# PyFIM - Coop Case Study - Complementary and Substitute Products 

## Complementary and Substitute Products Products PyFIM

## Preprocessing

In [198]:
# Libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pylab as pylab
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error as mse 
from sklearn.neural_network import MLPRegressor
from apyori import apriori ## Importing apriori package
from fim import arules
#from mlxtend.frequent_patterns import apriori, association_rules

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (22, 14),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

pylab.rcParams.update(params)

#### Import functions from ./Functions_Forecasting AR file in this same folder, look for comments on the functions in that notebook



In [199]:
from ipynb.fs.defs.Functions_Forecasting import * 


### importing data

In [200]:
data = pd.read_csv("./4months_v2/rl_4months.csv")

In [201]:
data.shape

(3221229, 40)

In [202]:
#data[["receiptKey","DayDate","hourOfDay","minuteOfHour","quantity","ItemID"]].iloc[0:20,]

In [203]:
# Generate unique name 
data['Unique_Item_Name'] = data['ItemCategoryName'] + '_' + data['ItemSegmentName']

In [204]:
data_subset = data[['receiptKey', 'DayDate','hourOfDay', 'ItemID', 'quantity','Unique_Item_Name']]


### Deletion of returns and false duplicates of Negative and 0 Quantities

In [205]:
data_subset = data_subset[data_subset.quantity>0]

In [206]:
data_subset['timestamp'] = data_subset['DayDate'].astype(str) + ' '+  data_subset['hourOfDay'].astype(str)


### Aggregating Datasets

In [207]:
data_subset_sub = data_subset[['receiptKey', 'ItemID']]
data_subset_sub

,receiptKey,ItemID
0,5623185059,277403
1,5623185059,322
3,5623185059,303321
4,5623185059,601635
5,5623185059,1246
...,...,...
3221218,5710791413,59422
3221222,5710791413,145269
3221224,5710791413,198267
3221226,5710791413,274768


In [208]:
transactions = [a[1]['ItemID'].tolist() for a in list(data_subset_sub.groupby(['receiptKey']))] ## Combing all the items in list format for each cutomer

In [209]:
hash_map_series = data_subset.groupby(['ItemID']).first()['Unique_Item_Name']

In [210]:
hash_map_series = pd.DataFrame(hash_map_series.str.strip())# Remove 

In [211]:
hash_map_series

,Unique_Item_Name
ItemID,
5,GODIS_STANDARD LÖSVIKT
7,LÖSVIKTSGODIS_STANDARD
8,NATURGODIS LÖSVI_NÖTTER
15,BAKNING_ÖVRIG FRÖ&KÄRNOR
16,FLINGOR_STANDARD M
...,...
804121,GODIS_0
805591,UTL. MATKONCEPT_0
805612,UTL. MATKONCEPT_0


In [212]:
??arules

## PyFIM

In [213]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 15 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [214]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [223]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [224]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(51, 5)


In [225]:
hash_map_series

,Unique_Item_Name
ItemID,
5,GODIS_STANDARD LÖSVIKT
7,LÖSVIKTSGODIS_STANDARD
8,NATURGODIS LÖSVI_NÖTTER
15,BAKNING_ÖVRIG FRÖ&KÄRNOR
16,FLINGOR_STANDARD M
...,...
804121,GODIS_0
805591,UTL. MATKONCEPT_0
805612,UTL. MATKONCEPT_0


In [229]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [230]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x: [hash_map_series.loc[i][0] for i in x])

In [231]:
results_df_names

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,46877,"(64582,)",4290,0.022887,28.493624,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_KOKBANANER]]
1,46877,"(62104,)",1998,0.010659,29.883338,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[GRÖNSAKER_GUL LÖK]]
2,46877,"(61858,)",1842,0.009827,48.885350,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_ÄPPLEN RÖDA]]
3,46877,"(61960,)",1799,0.009597,32.876462,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_APELSINER]]
4,46877,"(276012,)",1706,0.009101,34.492519,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[ÄGG_20 PACK ELLER STÖRRE]]
5,46877,"(62132,)",1647,0.008787,22.611203,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[GRÖNSAKER_LÖSVIKT]]
6,46877,"(278554,)",1592,0.008493,42.453333,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[FRUKT & BÄR_CITRONER]]
7,46877,"(273860,)",1539,0.008210,30.319149,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[GRÖNSAKER_MORÖTTER]]
8,46877,"(274461,)",1520,0.008109,30.919447,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[ÄGG_10-12 PACK]]
9,46877,"(306740,)",1032,0.005506,24.951644,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[[MJUKT MATBRÖD_FÖRÄDLAT]]


# Apriori Algorithm PyFIM Libary

In [232]:
from fim import eclat

In [240]:
# inputs
supp = 2 # minimum support of an assoc. rule (default: 10)
conf = 30 # minimum confidence of an assoc. rule (default: 80%)
report = 'asC'

In [241]:
result = eclat(transactions, supp=supp, conf=conf, report=report)

In [242]:
# make dict for nicer looking column names
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'xx',
    'S': 'support_emptyset',
    }

In [243]:
result = arules(transactions, supp=supp, conf=conf, report=report)

In [247]:
colnames = ['consequent', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(result, columns=colnames)
df_rules = df_rules.sort_values('support_itemset_absolute', ascending=False)
print(df_rules.shape)


(11, 5)


In [248]:
results_df_names = df_rules.merge(hash_map_series, left_on='consequent', right_on='ItemID').rename(columns= {'Unique_Item_Name': 'consequent names'})


In [249]:
results_df_names = results_df_names.dropna()

In [250]:
results_df_names['antecedent_names'] = results_df_names.antecedent.apply(lambda x:"" if x== () else [hash_map_series.loc[i][0] for i in x])

In [251]:
results_df_names 

,consequent,antecedent,support_itemset_absolute,support_itemset_relative,confidence_pct,consequent names,antecedent_names
0,64582,"(61960,)",2012,0.010734,36.769006,FRUKT & BÄR_KOKBANANER,[FRUKT & BÄR_APELSINER]
1,64582,"(61858,)",1756,0.009368,46.602972,FRUKT & BÄR_KOKBANANER,[FRUKT & BÄR_ÄPPLEN RÖDA]
2,64582,"(277670,)",1296,0.006914,30.587680,FRUKT & BÄR_KOKBANANER,[GRÖNSAKER_GURKA]
3,64582,"(278554,)",1221,0.006514,32.560000,FRUKT & BÄR_KOKBANANER,[FRUKT & BÄR_CITRONER]
4,46877,"(61858,)",1842,0.009827,48.885350,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[FRUKT & BÄR_ÄPPLEN RÖDA]
5,46877,"(61960,)",1799,0.009597,32.876462,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[FRUKT & BÄR_APELSINER]
6,46877,"(276012,)",1706,0.009101,34.492519,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[ÄGG_20 PACK ELLER STÖRRE]
7,46877,"(278554,)",1592,0.008493,42.453333,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[FRUKT & BÄR_CITRONER]
8,46877,"(273860,)",1539,0.008210,30.319149,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[GRÖNSAKER_MORÖTTER]
9,46877,"(274461,)",1520,0.008109,30.919447,BYGG&TRÄGÅRDSTJÄ_TJÄNSTER ÅTERKÖP ETC,[ÄGG_10-12 PACK]
